![A car dashboard with lots of new technical features.](images/dashboard.jpg)

You're working for a well-known car manufacturer who is looking at implementing LLMs into vehicles to provide guidance to drivers. You've been asked to experiment with integrating car manuals with an LLM to create a context-aware chatbot. They hope that this context-aware LLM can be hooked up to a text-to-speech software to read the model's response aloud.

As a proof of concept, you'll integrate several pages from a car manual that contains car warning messages and their meanings and recommended actions. This particular manual, stored as an HTML file, `mg-zs-warning-messages.html`, is from an MG ZS automobile, a compact SUV. Armed with your newfound knowledge of LLMs and LangChain, you'll implement Retrieval Augmented Generation (RAG) to create the context-aware chatbot.

**Note: Although we'll be using the OpenAI API in this project, you do not need to specify an API key.**

In [19]:
# Run this cell to install the necessary packages
import subprocess
import pkg_resources

def install_if_needed(package, version):
    '''Function to ensure that the libraries used are consistent to avoid errors.'''
    try:
        pkg = pkg_resources.get_distribution(package)
        if pkg.version != version:
            raise pkg_resources.VersionConflict(pkg, version)
    except (pkg_resources.DistributionNotFound, pkg_resources.VersionConflict):
        subprocess.check_call(["pip", "install", f"{package}=={version}"])

install_if_needed("langchain-core", "0.3.72")
install_if_needed("langchain-openai", "0.3.28")
install_if_needed("langchain-community", "0.3.27")
install_if_needed("unstructured", "0.18.11")
install_if_needed("langchain-chroma", "0.2.5")
install_if_needed("langchain-text-splitters", "0.3.9")

In [20]:
# Import the required packages
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [21]:
# Load the HTML as a LangChain document loader
loader = UnstructuredHTMLLoader(file_path="data/mg-zs-warning-messages.html")
car_docs = loader.load()

In [22]:
import os

# Load the models required to complete the exercise
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.environ["OPENAI_API_KEY"])

In [23]:
# Start coding here, use as many cells as you like
print(car_docs)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap=100,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
    ]
)

chunks = text_splitter.split_documents(
    documents=car_docs
)


for i, chunk in enumerate(chunks[:3]):
    print(f"Chunk {i+1}:\n{chunk.page_content}\n")

[Document(metadata={'source': 'data/mg-zs-warning-messages.html'}, page_content='Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.\n\nWarning Message Procedure Low Oil Pressure Indicates that the oil pressure is too low, which may result in severe engine damage. As s

In [24]:
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)

prompt= """
Use only the context given to answer the following question, if you are unsure, say it
Context: {context}
Question: {question}
"""


prompt_template = ChatPromptTemplate.from_template(prompt)

retreiver = vector_store.as_retriever(
   search_type="similarity", 
   search_kwargs={"k":2}
)

chain = ({"context":retreiver, "question":RunnablePassthrough()}
         | prompt_template 
         | llm
         )

question = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?" 

answer=chain.invoke(question)
print(answer)

content='The Gasoline Particular Filter Full warning indicates that the particular filter is full. You should consult an MG Authorised Repairer as soon as possible.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 233, 'total_tokens': 262, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CR4nqWiqEU3XKjWk4AIFsMM4wQNCY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--c49ea618-ee1a-4bf8-9c84-fea318e9efb3-0' usage_metadata={'input_tokens': 233, 'output_tokens': 29, 'total_tokens': 262, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
